# Mutation Analysis
Anteriormente, vimos como el coverage puede ayudarnos a identificar las partes de código ejecutadas por las pruebas. Sin embargo, el coverage solo puede que no sea la mejor medida para representar la effectividad de las pruebas. Debido a que no podría tener una buena covertura sin revisar si los resultados son correctos. Por ejemplo, ejecutamos una función con muchas entradas, y tenemos 100% de coverage pero nunca probamos si el resultado de la función es correcto.

Este capitulo esta 100% basado en el libro de Andreas Zeller, Rahul Gopinath, Marcel Böhme, Gordon Fraser, and Christian Holler, **The Fuzzing Book**, https://www.fuzzingbook.org/html/Fuzzer.html. Retrieved 2024-06-29 17:55:20+02:00.

## Pre-requisitos

In [1]:
from IPython.display import clear_output
!apt-get update
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz
!pip install fuzzingbook
from typing import Tuple, List, Callable, Set, Any
from urllib.parse import urlparse
clear_output()


In [2]:
import sys
from fuzzingbook.MutationAnalysis import MuFunctionAnalyzer
import fuzzingbook.bookutils.setup


## ¿Por qué el coverage estructural no es suficiente?

Uno de los problemas del coverage es que este no permite verificar si los resultados o los outputs de lo que estamos probando es correcto. Es decir, si una ejecución produce malos resultados, estos pasarían desapercibidos por la batería de pruebas.

Por ejemplo, si uno borra los asserts de las pruebas el test seguirá teniendo la misma cobertura, pero el test no estaría probando nada. Por ejemplo:

In [51]:
def ineffective_test_1():
    execute_the_program_as_a_whole()
    assert True

La assertion del final siempre pasa, sin importar que  execute_the_program_as_a_whole funcione. Claro, si la función execute_the_program_as_a_whole lanza exception entonces, si el test fallara, pero prodiamos tener un mejor test para esto:

In [52]:
def ineffective_test_2():
    try:
      execute_the_program_as_a_whole()
    except:
      pass
    assert True

El problema con estos test, sin embargo, es que la funcion execute_the_program_as_a_whole() puede obtener un coverage del 100%, pero no es efectivo para detectar bugs.



### Ejemplo: Limitaciones del Coverage

A continuación introduciremos ejemplos mas detallados para ilustrar la problematica del coverage y también como en analysis de mutación funciona. La función triangle clasifica si un triángulo con ejes de tamaño a, b y c calzan en una categoria de triángulos. Queremos testear si el programa funciona correctamente.

In [40]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'Equilateral'
        else:
            return 'Isosceles'
    else:
        if b == c:
            return "Isosceles"
        else:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"

Aqui tenemos un par de tests que prueba si la función funciona:

In [53]:
def strong_oracle(fn):
    assert fn(1, 1, 1) == 'Equilateral'

    assert fn(1, 2, 1) == 'Isosceles'
    assert fn(2, 2, 1) == 'Isosceles'
    assert fn(1, 2, 2) == 'Isosceles'

    assert fn(1, 2, 3) == 'Scalene'

Si ejecutamos los test estos pasan:

In [54]:

strong_oracle(triangle)


Sin embargo, la frase "todos los tests pasan" tiene valor solo si sabemos que nuestros test son efectivos. Pero, ¿qué hace a un test efectivo?
Revisemos el coverage de nuestros tests:

In [55]:
import fuzzingbook.bookutils.setup
from fuzzingbook.Coverage import Coverage
import inspect

Para facilitar el análisis, agregaremos a nuestra clase `Coverage` el método `show_coverage`:

In [56]:
class VisualCoverage(Coverage):
    def show_coverage(self, fn):
        src = inspect.getsource(fn)
        name = fn.__name__
        covered = set([lineno for method,
                       lineno in self._trace if method == name])
        for i, s in enumerate(src.split('\n')):
            print('%s %2d: %s' % ('#' if i + 1 in covered else ' ', i + 1, s))

In [58]:
with VisualCoverage() as cov:
    strong_oracle(triangle)


In [59]:

cov.show_coverage(triangle)


   1: def triangle(a, b, c):
#  2:     if a == b:
#  3:         if b == c:
#  4:             return 'Equilateral'
   5:         else:
#  6:             return 'Isosceles'
   7:     else:
#  8:         if b == c:
#  9:             return "Isosceles"
  10:         else:
# 11:             if a == c:
# 12:                 return "Isosceles"
  13:             else:
# 14:                 return "Scalene"
  15: 


`strong_oracle` parece tener un coverage adecuado que cubre todas las posibles condiciones. Eso es verdad, y los test se ven razonablemente buenos en terminos coverage.

Sin embargo, ¿la cobertura nos dice todo sobre nuestros tests?

Consideremos el siguiente conjunto de tests:

In [60]:
def weak_oracle(fn):
    assert fn(1, 1, 1) == 'Equilateral'

    assert fn(1, 2, 1) != 'Equilateral'
    assert fn(2, 2, 1) != 'Equilateral'
    assert fn(1, 2, 2) != 'Equilateral'

    assert fn(1, 2, 3) != 'Equilateral'

Para este test que ve si un triángulo tiene diferentes lados no es equilatero. ¿Cuál es el coverage?

In [61]:
with VisualCoverage() as cov:
    weak_oracle(triangle)

cov.show_coverage(triangle)


   1: def triangle(a, b, c):
#  2:     if a == b:
#  3:         if b == c:
#  4:             return 'Equilateral'
   5:         else:
#  6:             return 'Isosceles'
   7:     else:
#  8:         if b == c:
#  9:             return "Isosceles"
  10:         else:
# 11:             if a == c:
# 12:                 return "Isosceles"
  13:             else:
# 14:                 return "Scalene"
  15: 


Se puede observar que esta batería de tests tiene la misma cobertura que la anterior.

Reflexionando un poco `weak_oracle` no es tan efectivo como `strong_oracle`.

Sin embargo, en terminos de coverage ambos parecen tener la misma calidad. Pero esto **no es cierto**. Por eso, es necesario también evaluar la calidad de los assertions.

Entonces, ¿cómo puedo saber si mis test en verdad son útiles en detectar bugs? una alternativa es **inyectar bugs dentro del programa y evaluar la effectividad de nuestras pruebas en atraparlos.**

Pero, ¿cómo producimos bugs en primer lugar? si ponemos **bugs de forma manual**, estos estarían **sesgados** a la perspectiva del desarrollador sobre el como la aplicación puede funcionar. Es más, escribir buenos bugs involucra también mucho tiempo.

## Sembrando bugs artificales con Mutation Analysis

Mutation Analysis ofrece una solucion alternativa para evaluar la efectividad de los test generados.

La idea del mutation analysis es sembrar bugs artificiales, conocidos como mutantes, dentro del código del programa, y verficar cuando las pruebas la encuentran.

Una mutación podría, por ejemplo, reemplazar un + por un - (en algún lugar del programa). Por su puesto, un conjunto de test ineficientes no detectarían este cambio.


In [62]:
def suma_generica(a, b):
  return a + b

def suma_generica_mutante(a, b):
  return a - b


La hipótesis es que si se pueden detectar estos bugs artificiales, habrían más chances de detectar bugs reales.

El objetivo del Mutation Analysis es considerar la probabilidad de insertar un bug desde la perspectiva de un programador. Si uno asume que la atención recibida por cada elemento del programa es similar, uno podría asumir que cada token del programa tiene una misma probabilidad de contener un bug.

Por su puesto, un programador puede corregir cualquier error que es detectado por el compilador o por un linter. Entonces, para establecer tokens válidos en diferentes partes del programa, que logren pasar la compilación, pueden ser considerados posibles fallas del programa.

Una batería de pruebas es un juez capas de detectar estos mutantes. El mutation score, representa la proporción de mutantes detectados sobre todos los mutantes válidos producidos.

## Inyectando Fallas Artificiales

Lo que necesitamos hacer es injectar bugs en el programa, uno a la vez, y contar cuantos de estos bugs los test logran detectar. La frecuencia sera un indicador de cuantos bugs fueron no cubiertos. A esta técnica se llama fault injection.

In [14]:
def triangle_m1(a, b, c):
    if a == b:
        if b == c:
            return 'Equilateral'
        else:
            # return 'Isosceles'
            return None  # <-- injected fault
    else:
        if b == c:
            return "Isosceles"
        else:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"

Veamos si los test son buenos para detectar este bug.

In [63]:
from fuzzingbook.ExpectError import ExpectError

In [65]:
with ExpectError():
    weak_oracle(triangle_m1)


Los test en weak_oracle no son capaces de detectar ningun cambio.

In [66]:
with ExpectError():
    strong_oracle(triangle_m1)


Traceback (most recent call last):
  File "<ipython-input-66-cf9d4456bba5>", line 2, in <cell line: 1>
    strong_oracle(triangle_m1)
  File "<ipython-input-53-90155baa3232>", line 5, in strong_oracle
    assert fn(2, 2, 1) == 'Isosceles'
AssertionError (expected)


Nuestra `strong_oracle` es capaz de detectar la falla, esto evidencia que `strong_oracle` es probablemente el mejor conjunto de tests.

Fault injection puede proporcionar una buena forma de medir la efectividad de un test suite. Sin embargo, es difícil crear un conjunto de "buenos" faults que sean razonablemente dificiles de detectar. Además, este es un proceso manual.

Por otro lado, dado que la inyección fue manual, el proceso de generar fallas **estará sesgado a la percepción del desarrollador que crea los bugs**, incluso si le pasamos una lista de bugs a inyectar esto sería un proceso tedioso. ¿Podemos hacerlo mejor?


El Mutation Analysis proporciona una alternativa. La idea es que, si uno asume que el programador entiende el programa en cuestion, la mayoría de los errores realizados deberían ser similares a errores de transcripción (solo un par de palabras).

Un compilador podrá detectar la mayoría de estos errores. Por lo que la mayoría de fallas sobrantes en el programa son probablemente debido a una variación pequeña en ciertos puntos de la estructura del programa.

Esto también es llamado "Competent Programmer Hypothesis" o "Finite Neighborhood Hypothesis".

Pero, ¿qué pasa con las fallas grandes que estan compuestas por fallas pequeñas? La idea aquí es que, para la mayoría de las fallas complejas, los casos de prueba que detecten una falla simple de manera aislada podrán muy probablemente detectar fallas grandes. Esta suposición es llamada "Coupling Effect".

¿Cómo podemos utilizar esta suposición en la práctica? La idea no es simplemente generar todas las posbiles variantes validad de un programa que difieren del original con un pequeño cambio.

Cualquier mutante detectado por la batería de pruebas se dice que fue matado por la batería. **La efectivdad esta dada por la proporción de mutantes muertos con respecto a todos los mutantes validos generados.**

## Importando y adaptando una simple mutación

En este capítulo utilizaremos una clase que nos permite realizar mutaciones. La siguiente parte del curso, veremos como implementar este tipo de clases.

Cabe notar que en el siguiente bloque de código no solo importa la clase que nos permite hacer mutaciones sino que también aplica unos parches para que esta funcione en google colab.

In [68]:
from fuzzingbook.MutationAnalysis import Mutant
from fuzzingbook.MutationAnalysis import MuFunctionAnalyzer

# parche 1
def __fixed_ented__(self):
  if self.log:
    print('->\t%s' % self.name)
  exec(self.src(), globals())

# parche 2
def __fixed_exit__(self, exc_type, exc_value, traceback):
  if self.log:
      print('<-\t%s' % self.name)
  if exc_type is not None:
      self.detected = True
      if self.log:
          print("Detected %s" % self.name, exc_type, exc_value)
  globals()[self.pm.name] = self.pm.fn
  if self.log:
      print()
  return True

# aplicando parches
Mutant.__enter__= __fixed_ented__
Mutant.__exit__= __fixed_exit__

## Mutating Python Code

En las siguientes clases veremos como generar los mutantes nosotros mediante un algoritmo. De momento nos limitaremos a utilizar la clase del fuzzingbook que ya injecta los mutantes y ejecuta un pedazo de código enviando varias versiones mutadas de la función orignial.

In [19]:
import difflib

for mutant in MuFunctionAnalyzer(triangle):
    print(mutant.diff())

--- original

+++ mutant

@@ -1,7 +1,7 @@

 def triangle(a, b, c):
     if a == b:
         if b == c:
-            return 'Equilateral'
+            pass
         else:
             return 'Isosceles'
     elif b == c:
--- original

+++ mutant

@@ -3,7 +3,7 @@

         if b == c:
             return 'Equilateral'
         else:
-            return 'Isosceles'
+            pass
     elif b == c:
         return 'Isosceles'
     elif a == c:
--- original

+++ mutant

@@ -5,7 +5,7 @@

         else:
             return 'Isosceles'
     elif b == c:
-        return 'Isosceles'
+        pass
     elif a == c:
         return 'Isosceles'
     else:
--- original

+++ mutant

@@ -7,6 +7,6 @@

     elif b == c:
         return 'Isosceles'
     elif a == c:
-        return 'Isosceles'
+        pass
     else:
         return 'Scalene'
--- original

+++ mutant

@@ -9,4 +9,4 @@

     elif a == c:
         return 'Isosceles'
     else:
-        return 'Scalene'
+        pass


In [69]:
import sys
for mutant in MuFunctionAnalyzer(triangle,log=True):
    with mutant:
      assert triangle(1, 1, 1) == 'Equilateral', "Equal Check1"
      assert triangle(1, 0, 1) != 'Equilateral', "Equal Check2"
      assert triangle(1, 0, 2) != 'Equilateral', "Equal Check3"

mutant.pm.score()

->	triangle_1
<-	triangle_1
Detected triangle_1 <class 'AssertionError'> Equal Check1

->	triangle_2
<-	triangle_2

->	triangle_3
<-	triangle_3

->	triangle_4
<-	triangle_4

->	triangle_5
<-	triangle_5



0.2

El código anterior, creo 5 versiones de la función `triangle`, cada una con una mutación, y ejecutó el bloque de código anterior con cada una de estas versions, es decir, la función `triangle`, en cada iteración era una version-mutada.

Solo una de las cinco mutaciones resulto en un assert que falle (failing assertion). **Por lo que los test solo lograron matar al 20% de los mutantes.**

In [70]:
for mutant in MuFunctionAnalyzer(triangle):
    with mutant:
        weak_oracle(triangle)
mutant.pm.score()

0.2

In [71]:
for mutant in MuFunctionAnalyzer(triangle):
    with mutant:
        strong_oracle(triangle)
mutant.pm.score()

1.0

La métrica mutation score es más acorde a la calidad de los assertions del `weak_oracle` y `strong_oracle`.

Veamos otro ejemplo:

In [23]:
def gcd(a, b):
    if a < b:
        c = a
        a = b
        b = c

    while b != 0:
        c = a
        a = b
        b = c % b

    return a

In [72]:
for mutant in MuFunctionAnalyzer(gcd, log=True):
    with mutant:
        assert gcd(1, 0) == 1, "Minimal"
        assert gcd(0, 1) == 1, "Mirror"

mutant.pm.score()

->	gcd_1
<-	gcd_1
Detected gcd_1 <class 'UnboundLocalError'> local variable 'c' referenced before assignment

->	gcd_2
<-	gcd_2
Detected gcd_2 <class 'AssertionError'> Mirror

->	gcd_3
<-	gcd_3

->	gcd_4
<-	gcd_4

->	gcd_5
<-	gcd_5

->	gcd_6
<-	gcd_6

->	gcd_7
<-	gcd_7
Detected gcd_7 <class 'AssertionError'> Minimal



0.42857142857142855

## Mutation Testing con Unit Tests

In [25]:
import unittest

In [26]:
class StrongShapeTest(unittest.TestCase):

    def test_equilateral(self):
        assert triangle(1, 1, 1) == 'Equilateral'

    def test_isosceles(self):
        assert triangle(1, 2, 1) == 'Isosceles'
        assert triangle(2, 2, 1) == 'Isosceles'
        assert triangle(1, 2, 2) == 'Isosceles'

    def test_scalene(self):
        assert triangle(1, 2, 3) == 'Scalene'

In [27]:
def suite(test_class):
    suite = unittest.TestSuite()
    for f in test_class.__dict__:
        if f.startswith('test_'):
            suite.addTest(test_class(f))
    return suite

In [28]:
suite(StrongShapeTest).run(unittest.TestResult())


<unittest.result.TestResult run=3 errors=0 failures=0>

In [73]:
with VisualCoverage() as cov:
    suite(StrongShapeTest).run(unittest.TestResult())

cov.show_coverage(triangle)


   1: def triangle(a, b, c):
#  2:     if a == b:
#  3:         if b == c:
#  4:             return 'Equilateral'
   5:         else:
#  6:             return 'Isosceles'
   7:     else:
#  8:         if b == c:
#  9:             return "Isosceles"
  10:         else:
# 11:             if a == c:
# 12:                 return "Isosceles"
  13:             else:
# 14:                 return "Scalene"
  15: 


In [30]:
class WeakShapeTest(unittest.TestCase):
    def test_equilateral(self):
        assert triangle(1, 1, 1) == 'Equilateral'

    def test_isosceles(self):
        assert triangle(1, 2, 1) != 'Equilateral'
        assert triangle(2, 2, 1) != 'Equilateral'
        assert triangle(1, 2, 2) != 'Equilateral'

    def test_scalene(self):
        assert triangle(1, 2, 3) != 'Equilateral'

In [31]:
with VisualCoverage() as cov:
    suite(WeakShapeTest).run(unittest.TestResult())

cov.show_coverage(triangle)


   1: def triangle(a, b, c):
#  2:     if a == b:
#  3:         if b == c:
#  4:             return 'Equilateral'
   5:         else:
#  6:             return 'Isosceles'
   7:     else:
#  8:         if b == c:
#  9:             return "Isosceles"
  10:         else:
# 11:             if a == c:
# 12:                 return "Isosceles"
  13:             else:
# 14:                 return "Scalene"
  15: 


En este capitulo vimos como ejecutar mutation testing sobre funciones, si quieres aprender como implementar una clase para evaluar classes y modulos de python, y analizar la calidad de unit tests. Se le recomienda leer el libro fuzzing book "Lexical Fuzzing">>"Mutation Analysis"

## Práctica

In [74]:
def describe_shape(sides, side_lengths=None):
    if sides < 3:
        return "No es una figura geométrica válida"

    if sides == 3:
        if side_lengths and len(set(side_lengths)) == 1:
            return "Es un triángulo equilátero"
        elif side_lengths and len(set(side_lengths)) == 2:
            return "Es un triángulo isósceles"
        else:
            return "Es un triángulo escaleno"

    elif sides == 4:
        if side_lengths and len(set(side_lengths)) == 1:
            return "Es un cuadrado"
        elif side_lengths and len(set(side_lengths)) == 2:
            if side_lengths.count(side_lengths[0]) == 2:
                return "Es un rectángulo"
            else:
                return "Es un rombo"
        else:
            return "Es un cuadrilátero irregular"

    else:
        return "Figura geométrica con más de 6 lados o no es una figura regular"


import sys
for mutant_excercise in MuFunctionAnalyzer(describe_shape, log=True):
    with mutant_excercise:
      assert describe_shape(2) == 'No es una figura geométrica válida', "Test 1"
      assert describe_shape(3, [1,2,1]) == 'Es un triángulo isósceles', "Test 2"

mutant_excercise.pm.score()

->	describe_shape_1
<-	describe_shape_1
Detected describe_shape_1 <class 'AssertionError'> Test 1

->	describe_shape_2
<-	describe_shape_2

->	describe_shape_3
<-	describe_shape_3
Detected describe_shape_3 <class 'AssertionError'> Test 2

->	describe_shape_4
<-	describe_shape_4

->	describe_shape_5
<-	describe_shape_5

->	describe_shape_6
<-	describe_shape_6

->	describe_shape_7
<-	describe_shape_7

->	describe_shape_8
<-	describe_shape_8

->	describe_shape_9
<-	describe_shape_9



0.2222222222222222

## Revisión de los mutantes creados

In [33]:
for mutant_excercise in MuFunctionAnalyzer(describe_shape):
    print(mutant_excercise.diff())

--- original

+++ mutant

@@ -1,6 +1,6 @@

 def describe_shape(sides, side_lengths=None):
     if sides < 3:
-        return 'No es una figura geométrica válida'
+        pass
     if sides == 3:
         if side_lengths and len(set(side_lengths)) == 1:
             return 'Es un triángulo equilátero'
--- original

+++ mutant

@@ -3,7 +3,7 @@

         return 'No es una figura geométrica válida'
     if sides == 3:
         if side_lengths and len(set(side_lengths)) == 1:
-            return 'Es un triángulo equilátero'
+            pass
         elif side_lengths and len(set(side_lengths)) == 2:
             return 'Es un triángulo isósceles'
         else:
--- original

+++ mutant

@@ -5,7 +5,7 @@

         if side_lengths and len(set(side_lengths)) == 1:
             return 'Es un triángulo equilátero'
         elif side_lengths and len(set(side_lengths)) == 2:
-            return 'Es un triángulo isósceles'
+            pass
         else:
             return 'Es un triángulo escal